# Segmenting and Clustering Neighborhoods in Toronto

**Harry Ngo**   

**_Background_ _information_**  
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.   
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.  
Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## Step 1: Read in and clean Wikipedia data

In [2]:
import pandas as pd

In [56]:
# Using pandas to read HTML
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [57]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'.
df = df[df['Borough'] != 'Not assigned']
# Rename Postal code column
df = df.rename(columns={'Postal code': 'PostalCode'})

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [58]:
# More than one neighborhood can exist in one postal code area, separated by commas.
df['Neighborhood'] = (df['Neighborhood'].replace('\/',',', regex=True)
                        .replace(' ,',',', regex=True) # removes whitespace before comma/s
                        .astype(str))
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [59]:
# If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough.
df['Neighborhood'].replace('Not assigned', df['Borough'], inplace=True)

In [60]:
# Use the .shape method to print the number of rows of your dataframe
df.shape

(103, 3)

## Step 2: Add geospatial data to the dataframe

In [61]:
# Read in geospatial data
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
# Rename Postal Code column
geo_df = geo_df.rename(columns={'Postal Code': 'PostalCode'})

geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [64]:
# Merge the dataframes on common key PostalCode
df_merged = pd.merge(df, geo_df, on='PostalCode')
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
